In [39]:
using GLMakie, LinearAlgebra

include("boids.jl")

# TODO: Make boids not collided
# TODO: Make border limits for boids
# TODO: Update buttons, step and restart
# TODO: Try to make inifite run/stop animation
# TODO: Maybeee sliders for parameters?
# TODO: Maybeeeeee try make .exe 

# Make boids
width = height = 500;
numBoids = 1000
flock = [Boid(i, rand(1:width, 3), randn(3).*2, randn(3)) for i=1:numBoids]
points = Observable([Point3f(boid.position) for boid in flock])

# Parameters
alignValue = 0.5# ls1[1].value
separationValue = 1#ls2[1].value
cohesionValue = 0.6 #ls3[1].value

# Create figure
fig = Figure()

# create 3d axis
ax = Axis3(fig[1, 1], aspect = (1, 1, 1),
        title = "3d scatter")
        ax.perspectiveness = 0

# create button grid
fig[2, 1] = buttongrid = GridLayout(tellwidth = false)

# Set limits
limits!(ax, -4*width, 4*width, -4*width, 4*width, -4*width, 4*width)

labels = ["Run", "Step", "Restart"]
button_count = 3

# create buttons
buttons = buttongrid[1, 1:button_count] = [
    Button(fig,
        label = l, height = 60, width = 250, textsize = 30
    )
    for l in labels
]

# run button
on(buttons[1].clicks) do click
    frames = 1:100
    @async begin
    for i = frames
        # points[] += Point3f[(0., 0.01, 0.)]
        for boid in flock
            fly!(boid, flock, alignValue[], separationValue[], cohesionValue[])
        end
        
        points[] = [Point3f(boid.position) for boid in flock]
        sleep(0.001)
    end
    end
    # record(fig, "append_animation.mp4", frames;
    #         framerate = 30) do frame
    #     points[] += Point3f[(0., 0.01, 0.)]
    #     sleep(0.05)
    # end
end

# step button - the same as run
on(buttons[2].clicks) do click
    points[] += Point3f[(0., 0.01, 0.)]
end

# reset button
on(buttons[3].clicks) do click
    points[] = Point3f[(0., 0., 0.)]
end

scatter!(ax, points, markersize=3000)

display(fig)

GLMakie.Screen(...)

In [33]:
flock = [Boid(i, rand(1:width, 3), randn(3).*2, randn(3)) for i=1:3]

3-element Vector{Boid}:
 Boid(1, [107, 360, 206], [2.5014485102271586, 2.0150410724922647, -4.118153796125154], [1.1927480365766638, -0.09590367589001327, 1.7905485028005907])
 Boid(2, [225, 166, 25], [1.5064636699647969, -2.242383980444681, 1.7722680474045103], [0.23979623347228043, 1.4012944128351748, 0.4468207769761274])
 Boid(3, [386, 373, 351], [-5.370779751133057, 0.6799396447659722, -0.365157237410028], [-1.5200979469290112, -0.7637868186641218, 1.5741213458308978])

In [34]:
alignValue = 0.5# ls1[1].value
separationValue = 1#ls2[1].value
cohesionValue = 0.6 #ls3[1].value

for boid in flock
    fly!(boid, flock, alignValue[], separationValue[], cohesionValue[])
end

In [35]:
flock

3-element Vector{Boid}:
 Boid(1, [109.50144851022716, 362.01504107249224, 201.88184620387486], [3.098197618464578, 1.609515583788768, -1.9520837846981673], [0.0, -0.0, 0.0])
 Boid(2, [226.5064636699648, 163.75761601955531, 26.77226804740451], [1.7462599034370774, -0.841089567609506, 2.2190888243806377], [0.0, 0.0, 0.0])
 Boid(3, [380.62922024886694, 373.67993964476597, 350.63484276259], [-3.9395414629219276, -0.047935753991611485, 0.6911694621496325], [-0.0, -0.0, 0.0])